---
title: "Build Model"
---

In [ ]:
using Atlans
using Dates

# Build standard Model with all processes
Below, an explanation is given how an Atlantis subsurface model is built

In [ ]:
basedir = "directory_to_your_project" 

path_to_subsurface_model = joinpath(basedir, "my_subsurface_model.nc")
path_to_parameter_table = joinpath(basedir, "my_parameter_table.nc")

model = Atlans.Model(
    Atlans.HydrostaticGroundwater,
    Atlans.DrainingAbcIsotache,
    Atlans.CarbonStore,
    Atlans.OverConsolidationRatio,
    Atlans.NullShrinkage,
    Atlans.AdaptiveCellsize(Δzmax, 0.01),
    Atlans.ExponentialTimeStepper(1.0, 2),
    path_to_subsurface_model,
    path_to_parameter_table,
)
